In [2]:
from bs4 import BeautifulSoup
import requests
from flask import request,render_template,Flask
from camelot import read_pdf
import html
import pandas as pd

In [3]:
app = Flask(__name__)

@app.route('/',methods=['GET'])
def index():
    return render_template('Home.html')

@app.route('/dashboard',methods=['GET'])
def today():
    response=requests.get("http://hmfw.ap.gov.in/covid_dashboard.aspx")
    soup=BeautifulSoup(response.text,'html.parser')
    table=soup.find('table',{'class':"table table-bordered table-striped"})
    tr_tags=table.find_all('tr')
    flag=0
    for tr_tag in tr_tags:
        td_tags=tr_tag.find_all('td')
        list_elements=[]
        for td_tag in td_tags:
            list_elements.append(td_tag.text.replace('\n',''))
        if flag==0:
            content=pd.DataFrame(columns=list_elements)
            flag=1
        else:
            district_row=pd.Series(list_elements,index=content.columns)
            content=content.append(district_row,ignore_index=True)
    table_html=content.to_html(columns=content.columns).replace('\n','')
    return render_template('table.html',dashboard=True,table=table_html)

@app.route('/andhrapradesh',methods=['POST','GET'])
def ap():
    date=request.form.get('date')
    url='http://hmfw.ap.gov.in/covid_19_datewisebulletins.aspx'
    payload={'date':date}
    response=requests.get(url,params=payload)
    soup=BeautifulSoup(response.text,'html.parser')
    for a in soup.find_all('a')[::-1]:
        if date in a.get('href'):
            url='http://hmfw.ap.gov.in'+a.get('href')
            break
    table=read_pdf(url)
    df=table[0].df
    df=df.rename(columns=df.iloc[0])[1:]
    table_html=df.to_html(columns=df.columns).replace('\n','')
    return render_template('table.html',dashboard=False,title=date,table=table_html)

In [4]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
